In [2]:
import pandas as pd
from time import strftime,localtime
from sqlalchemy import create_engine
from os import environ

In [3]:
from sys import path

In [4]:
path.insert(0,'./code/')

In [5]:
from web_scraper import *

In [6]:
from selenium import webdriver

In [6]:
def add_new_players(engine, names, ids):
    condition = str(tuple(ids))
    existing_players = pd.read_sql("select * from Players where ID IN "+condition,engine)
        
    c = pd.DataFrame({"ID":ids,"Name":names}).drop_duplicates()
    e = c.merge(existing_players,on="ID",how='left',indicator=True,suffixes = ('','_y'))
    e = e.loc[e["_merge"]== "left_only"][["ID","Name"]]
        
    e.to_sql("Players",engine,index=False, if_exists="append")
    
    return

In [7]:
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from re import search

from bs4 import BeautifulSoup

import pandas as pd

In [23]:
class Team_Box:
    def __init__(self):
        self.boxes = Team_box_scores()        
        self.table = "Team_box_scores"
        self.scores =[]
#         self.page_count = pd.read_csv("../box_score.csv")
        self.engine = create_engine("mariadb+mariadbconnector://"\
                                  +environ.get("USER")+":"\
                                  +environ.get("PSWD")+"@127.0.0.1:3306/nba")
        
        self.db_columns = ['Team_ID', 'Game_ID', 'Matchup', 'Game_day', 'Result',
                        'MINS', 'PTS', 'FGM', 'FGA', 'FGP', 'PM3', 'PA3', 'P3P', 'FTM', 'FTA',
                        'FTP', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV',  'PF']

        
    def add_new_box_scores(self,df):
        existing = pd.read_sql("select * from "+self.table,self.engine)

        e = df.merge(existing,on=["Team_ID","Game_ID"],how='left',indicator=True,suffixes = ('','_y'))
        e = e.loc[e["_merge"]== "left_only"][df.columns]

        e.to_sql(self.table,self.engine,index=False, if_exists="append")

        return

    def write(self,df,tids,gids):
        df.insert(0, "Team_ID",tids)
        df.insert(1, "Game_ID",gids)
        
                
        df = df[self.db_columns]
        df = df.drop_duplicates()
        df = df.dropna(subset = ['Result'])
        self.add_new_box_scores(df)
        
        return
    
    def preprocess(self,html):
        df = pd.read_html(html,na_values=['-'])[0]
        df = df.drop(columns = ['Season','+/-'])
        
        df = df[df.columns[1:]]
        
        d = dict(zip(df.columns,self.db_columns[2:]))
        df = df.rename(columns=d)
        df['Game_day'] = pd.to_datetime(df['Game_day'])
        
        return df

        
    
    def get_last_year(self,year):
        if(year > 2000):
            return year % 2000
        return year % 100
        
    def get_season(self, driver, year, reg_season=True):
        
        url = self.boxes.build_url(year,reg_season)
        
        for html in self.boxes.iter_all(url, driver):
            tids, gids = self.boxes.get_player_and_team_ids(html)
            df = self.preprocess(html)
            if self.max_date['GD'].max() > df['Game_day'].max():
                break
                
            self.write(df,tids,gids)
        
        return
        
       
    def get_team_stats(self,year,reg_season = True):
        with webdriver.Chrome() as driver:
            year_range = str(year-1) + "-{:0>2d}".format(self.get_last_year(year))

            self.get_season(driver,year_range,reg_season)

        return

In [24]:
t = Team_Box()

In [25]:
# t.get_team_stats(2013,True)

In [16]:
# for i in range(2014,2022):
#     for j in (True,False):
#         print(i,j)
#         t.get_team_stats(i,j)

In [12]:
class Box:
    def __init__(self):
        self.boxes = Box_scores()        
        self.table = "Box_scores"
        self.scores =[]
        command = 'SELECT MAX(Game_day) as GD from Box_scores'

        self.engine = create_engine("mariadb+mariadbconnector://"\
                                  +environ.get("USER")+":"\
                                  +environ.get("PSWD")+"@127.0.0.1:3306/nba")

        self.max_date = pd.read_sql(command,self.engine,parse_dates=['GD'])
        
        self.db_columns = ['Player_ID', 'Team_ID', 'Game_ID', 'Matchup', 'Game_day', 'Result',
                        'MINS', 'PTS', 'FGM', 'FGA', 'FGP', 'PM3', 'PA3', 'P3P', 'FTM', 'FTA',
                        'FTP', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV',  'PF']

    def add_new_players(self, names, ids):
        condition = str(tuple(ids))
        existing_players = pd.read_sql("select * from Players where ID IN "+condition,self.engine)
            
        c = pd.DataFrame({"ID":ids,"Name":names}).drop_duplicates()
        e = c.merge(existing_players,on="ID",how='left',indicator=True,suffixes = ('','_y'))
        e = e.loc[e["_merge"]== "left_only"][["ID","Name"]]
            
        e.to_sql("Players",self.engine,index=False, if_exists="append")
        
        return

    def add_new_box_scores(self,df):
        existing = pd.read_sql("select * from Box_scores",self.engine)
        
        e = df.merge(existing,on=["Player_ID","Team_ID","Game_ID"],how='left',indicator=True,suffixes = ('','_y'))
        e = e.loc[e["_merge"]== "left_only"][df.columns]
        
        e.to_sql("Box_scores",self.engine,index=False, if_exists="append")
        
        return   

    def write(self,df):        
                
        df = df[self.db_columns]
        df = df.drop_duplicates()
        df = df.dropna(subset = ['Result'])
        self.add_new_box_scores(df)
        
        return
    
    def preprocess(self,html, pids, tids, gids):
        df = pd.read_html(html,na_values=['-'])[0]
        df = df.drop(columns = ['Season','+/-','FP'])

        self.add_new_players(df[df.columns[0]],pids)
        df = df[df.columns[2:]]

        d = dict(zip(df.columns,self.db_columns[3:]))
        df = df.rename(columns=d)
        
        df.insert(0, "Player_ID",pids)
        df.insert(1, "Team_ID",tids)
        df.insert(2, "Game_ID",gids)
        df['Game_day'] = pd.to_datetime(df['Game_day'])
        
        
        return df

        
    def get_season(self, driver, year, reg_season=True):
        
        url = self.boxes.build_url(year,reg_season)
        df = pd.DataFrame()
        
        for html in self.boxes.iter_all(url, driver):
            with open('test.html','w') as f:
                f.write(html)
            pids, tids, gids = self.boxes.get_player_and_team_ids(html)
            df = pd.concat((df,self.preprocess(html,pids, tids, gids)))  

            if self.max_date['GD'].max() > df['Game_day'].max():
                break

                
        self.write(df)
        
        return
         
       
    def get_player_stats(self,year,reg_season = True):
        with webdriver.Chrome() as driver:
            year_range = str(year-1) + "-{:0>2d}".format(self.get_last_year(year))

            self.get_season(driver,year_range,reg_season)

        return

    @staticmethod
    def get_last_year(year):
        if(year > 2000):
            return year % 2000
        return year % 100

In [21]:
class Team_standings:
    
    def __init__(self):
        self.REG = "002"
        self.POST = "004"
        self.teams = Teams()
        self.db_columns = ['SEASON_ID', 'TEAM_ID', 'GP', 'W', 'L', 'WINP', 'MINS', 'PTS', 'FGM',
                        'FGA', 'FGP', 'PM3', 'PA3', 'P3P', 'FTM', 'FTA', 'FTP', 'OREB', 'DREB',
                        'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD']
        self.engine = create_engine("mariadb+mariadbconnector://"\
                                  +environ.get("USER")+":"\
                                  +environ.get("PSWD")+"@127.0.0.1:3306/nba")
        
    def get_last_year(self,year):
        if(year > 2000):
            return year % 2000
        return year % 100

    
    def write(self,html,tids,season_id):
        df = pd.read_html(html)[0]
        df = df.drop(columns = ['TEAM','+/-','Unnamed: 0','Unnamed: 1'])
#         df = df.dropna('columns')
        
        d = dict(zip(df.columns,self.db_columns[2:]))
        df = df.rename(columns=d)
        
        df["TEAM_ID"] = tids
        df["SEASON_ID"] = season_id
        
        
        df = df[self.db_columns]
        df = df.drop_duplicates()
        
        df.to_sql("Team_standings",self.engine,index=False, if_exists="append")
        
        return 

    def get_season(self, driver, year,season_id, reg_season=True):
        
        url = self.teams.build_url(year,reg_season)
        
        print(url)
        
        html,tids = self.teams.get_source_and_teams(url, driver)
        
        self.write(html,tids,season_id)
        
        return
    
    def get_team_standings(self,last_n_years):
        this_year = strftime("%Y",localtime())
        year = int(this_year) -  1
        
        with webdriver.Chrome() as driver:
            for y in range(year,year-last_n_years-1,-1):
                id_year = "{:0>2d}".format(self.get_last_year(y-1))
                
                select = "select SEASON_ID from Team_standings "
                condition = "where SEASON_ID LIKE '%" + id_year+"'"
                limit = " limit 1"
                
                query = select + condition +limit
                d = pd.read_sql(query,self.engine)
                if(len(d)== 0):# there is no data on this year, get it
                    year_range = str(y-1) + "-{:0>2d}".format(self.get_last_year(y))
                    season_id = self.REG + id_year
                    self.get_season(driver,year_range,season_id)

                    season_id = self.POST + id_year
                    self.get_season(driver,year_range,season_id,reg_season=False)

        
        return

In [1]:
class Player_stats:
    def __init__(self):
        self.players = Players()        
        self.REG = "002"
        self.POST = "004"
        self.db_columns = ['Player_ID', 'Team_ID', 'Age', 'GP', 'W', 'L', 'MINS', 'PTS', 'FGM',
                           'FGA', 'FGP', 'PM3', 'PA3', 'P3P', 'FTM', 'FTA', 'FTP', 'OREB', 'DREB',
                           'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'DD2', 'TD3', 'Season_ID']
        self.engine = create_engine("mariadb+mariadbconnector://"\
                                  +environ.get("USER")+":"\
                                  +environ.get("PSWD")+"@127.0.0.1:3306/nba")
        
        
    def write(self,html,pids,tids,season_id):
        df = pd.read_html(html)[0]
        
        df = df.drop(columns = ['PLAYER','TEAM','+/-'])
        df = df.dropna('columns')

        
        d = dict(zip(df.columns[1:],self.db_columns[2:-1]))
        df = df.rename(columns=d)
        
        
        df.insert(0,"Player_ID",pids)
        df.insert(1,"Team_ID",tids)
        df.insert(len(self.db_columns)-1,"Season_ID",season_id)
        
        df = df[self.db_columns]
        
        df.to_sql("Seasonal_performance",self.engine,index=False, if_exists="append")
        
        return 
    
    def get_last_year(self,year):
        if(year > 2000):
            return year % 2000
        return year % 100
        
    def get_season(self, driver, year,season_id, reg_season=True):
        
        url = self.players.build_url(year,reg_season)
        
        print(url)

        html = self.players.click_all(url, driver)
        pids, tids = self.players.get_player_and_team_ids(html)
        
        
        
        self.write(html,pids,tids,season_id)
        
        return
        
       
    def get_player_stats(self,last_n_years):
        year = int(strftime("%Y",localtime()))
        
        with webdriver.Chrome() as driver:
            for y in range(year,year-last_n_years-1,-1):
                id_year = "{:0>2d}".format(self.get_last_year(y-1))
                
                select = "select SEASON_ID from Seasonal_performance "
                condition = "where SEASON_ID LIKE '%" + id_year+"'"
                limit = " limit 1"
                
                query = select + condition +limit
                d = pd.read_sql(query,self.engine)

                if(len(d)== 0):# there is no data on this year, get it
                    year_range = str(y-1) + "-{:0>2d}".format(self.get_last_year(y))
                    season_id = self.REG + id_year
                    self.get_season(driver,year_range,season_id)

                    season_id = self.POST + id_year
                    self.get_season(driver,year_range,season_id,reg_season=False)

        return

In [13]:
bs = Box()

In [16]:
bs.get_player_stats(2022)

In [22]:
ts = Team_standings()

In [24]:
ts.get_team_standings(1)

https://www.nba.com/stats/teams/traditional/?Season=2021-22&SeasonType=Regular%20Season


/usr/lib/python3/dist-packages/ipykernel_launcher.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only


    GP   W   L   WIN%   MIN    PTS   FGM   FGA   FG%   3PM  ...  OREB  DREB  \
0   58  48  10  0.828  48.1  113.7  43.2  89.7  48.2  11.3  ...  10.1  36.1   
1   59  42  17  0.712  48.2  110.6  40.3  86.4  46.6  14.5  ...   9.7  36.1   
2   60  41  19  0.683  48.3  113.8  42.9  93.5  45.9  11.1  ...  13.9  35.0   
3   59  38  21  0.644  48.1  112.6  42.1  87.1  48.3  11.2  ...   8.9  34.4   
4   59  38  21  0.644  48.6  108.9  39.4  85.6  46.0  13.4  ...  10.8  34.0   
5   58  36  22  0.621  48.0  113.7  40.7  86.0  47.3  14.5  ...  10.2  35.9   
6   58  35  23  0.603  48.0  106.7  39.6  84.9  46.6  11.7  ...  10.3  34.5   
7   58  35  23  0.603  48.3  107.5  39.3  85.1  46.2  10.9  ...   8.6  33.8   
8   60  36  24  0.600  48.2  113.2  41.0  89.1  46.0  14.2  ...  10.2  36.2   
9   59  35  24  0.593  48.3  106.6  39.1  85.9  45.5  12.7  ...   9.5  34.2   
10  58  33  25  0.569  48.2  109.9  40.9  86.6  47.3  12.9  ...   9.2  34.8   
11  60  34  26  0.567  48.7  108.9  39.7  87.5  45.3

In [7]:
ps = Player_stats()

In [ ]:
ps.get_player_stats(1)